[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1dGiK2QKefEgE36tlitWT_ANkmZupzOcJ)

如有需要，请取消下行注释以安装 [geoca](https://haorui-jiang.github.io/geoca/)。

In [ ]:
# !pip install geoca

导入两个模块：**rasterconvert** 和 **cellularautomata**。

基于 **rasterconvert** 进行栅格数据格式转换，使用 **cellularautomata** 运行 CA 模型。

In [ ]:
from geoca import rasterconvert, cellularautomata

**rasterconvert.raster_list(file_path)**：

- 从文件中读取栅格数据，并将数据转换为 Python 列表，其中 NoData 值被转换为 None。

In [ ]:
raster_file = "Data/raster.tif"
data_list = rasterconvert.raster_list(raster_file)

# 读取数据列表的前 100 行，并输出所有非空值
for row in data_list[:100]:
    non_none_values = [value for value in row if value is not None]
    if non_none_values:
        print(non_none_values)

In [ ]:
# 设置原始栅格数据以及将要输出的新栅格数据文件路径
input_raster_path = raster_file
output_raster_path = "Data/result.tif"

**cellularautomata.migrate_population_focus(data_list, population, direction_num=4, proportion=1)**：

- 根据栅格像元值，将人口集中迁移至附近适宜性最强的区域。

**cellularautomata.migrate_population_disperse(data_list, population, direction_num=4, proportion=[0.5, 0.25, 0.15, 0.05])**：

- 根据栅格像素值，人口将 **分散** 迁移到各个邻近区域。

**rasterconvert.process_raster_data(input_raster_path, output_raster_path, new_data, nodata_value)**：

- 根据输入栅格创建栅格模板，并用新数据替换像素值来处理栅格数据。

如需在完成某些特定轮次的迭代后导出栅格数据时，可使用 **migrate_population_focus()** 或 **migrate_population_disperse()** 函数。

**注意：** 当使用 **migrate_population_disperse()** 函数时，由于人口向多个邻近位置迁移，因此不建议迭代次数过多，否则可能会导致人口过度分散。

In [ ]:
# 设置每个像元的初始种群数量为 10，运行 CA 模型进行特定次数迭代

# 创建与栅格数据列表大小相同的区域人口列表，并将所有列表元素统一设置为 10，代表每个栅格像元的初始人口规模
population = [[10 for _ in range(len(data_list[0]))] for _ in range(len(data_list))]

iterations = 20  # 设置迭代次数

for i in range(iterations):
    # 迁移 60% 的人口到附近适宜性最高的栅格像元
    population = cellularautomata.migrate_population_focus(data_list, population, 4, 0.6)

    # 对相邻的栅格像元适宜性进行排序，1/2 人口向最大值方向移动，1/4 人口向第二大方向移动，1/8 人口向第三大方向移动，其余人口留在原地
    population = cellularautomata.migrate_population_disperse(data_list, population, 4, [0.5, 0.25, 0.125, 0])
    print(f"Iteration {i + 1} is complete.")

    # 当迭代次数是 10 的倍数时，将迭代结果输出为栅格数据
    if (i + 1) % 10 == 0:
        # process_raster_data() 运行结束后，输出栅格的 NoData 值为 population 列表中的 0
        rasterconvert.process_raster_data(input_raster_path, f"Data/iterations_{i + 1}.tif", population, 0)

如需在完成所有迭代后导出栅格数据，请使用 **run_iterations_num()** 函数或 **run_iterations_pop()** 函数。

**cellularautomata.run_iterations_num(iterations, data_list, population_num=10, direction_num=4, type_migration="focus", migration_proportion=1)**：

- 使用 **统一的初始人口数** 运行元胞自动机，根据环境数据栅格模拟人口迁移。

In [ ]:
# 运行元胞自动机模拟 5 次迭代，每个栅格像元的初始人口数量为 10
new_data = cellularautomata.run_iterations_num(5, data_list, population_num=10)

rasterconvert.process_raster_data(input_raster_path, output_raster_path, new_data, 0)

**cellularautomata.run_iterations_pop(iterations, data_list, population_list, direction_num=4, type_migration="focus", migration_proportion=1)**：

- 基于 **初始人口数量栅格** 运行元胞自动机，根据环境数据栅格模拟人口迁移。

In [ ]:
population_file = "Data/population.tif"
population_list = rasterconvert.raster_list(population_file)

# 运行细胞自动机模拟 5 次迭代，每个栅格像元的初始人口大小由人口栅格值列表确定
new_data = cellularautomata.run_iterations_pop(5, data_list, population_list)

rasterconvert.process_raster_data(input_raster_path, output_raster_path, new_data, 0)

**cellularautomata.migrate_time(data_list, cost_list)**：

- 根据成本路径栅格及环境栅格计算迁移时间。

In [ ]:
cost_file = "Data/CostPath.tif"
cost_list = rasterconvert.raster_list(cost_file)

# 输出迁移时间、迭代次数及成本路径对应环境栅格值列表
time, count, costData_list = cellularautomata.migrate_time(data_list, cost_list)
print(time, count)
print(costData_list, len(costData_list))